### Importing libraries:

In [1]:
import pickle
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as ss
from pandas.plotting import scatter_matrix
import seaborn as sns
import pandas.testing as pdt
import scipy.stats as ss
import numpy.testing as npt
import pandas.testing as pdt

sns.set_style("darkgrid")

# <b>Title : Investigating factors determining property value in Mbombela </b>

### Compiled by:
1. Bongumusa Mavuso(1682836)
2. Siyabonga Hlomuka (1384685)
3. Thobelani Makeleni (1199116)

# <b>Executive Summary:</b>

The purpose of this notebook is to present a study that will investigate factors that influence the evaluation of the property value of the properties of the Mbombela Municipality.

The study will model the relationship between each of these different factors and the property value using the dataset from the property evaluation roll which was conducted in 2010 within the area of Mbombela by the Mbombela Municipality.

The main result of this study will be a linear model in this form -  $y = \beta_o + \beta_1 * x_1 + \beta_2* x_2 + ...$ $ where, \ \   x_i  \ \ where \ i\ = \ {{0, 1, ... , n}} \ $  represents the $ x^{ith}$ feature which influence the market value (y) of the property and $ \ \beta_i \ \ where \ i \ =  \ {0, 1, 2, .., n } \ $ is the weight of the $\ x^{ith} \$ feature.In addition we will also make use graphical representation that will show the relationships between the distinct features which have a direct causal influence in determining the market value of the properties,  and the market value of the properties.

#### Limitations:

We did not get a similar dataset to this one, which was collected prior 2010 or after 2010, which we can use to compare this dataset to or use it to test our model (test our model on a dataset from a different time period). The dataset was not accompanied by a data description hence we have no rich knowledge about the true meaning of some of the features , how the dataset was collected and how the properties used on the dataset were selected. Furthermore, the dataset is incomplete, since some of the features contained null values for the majority of their values. The data was only collected from Mbombela Mpumalanga, making it difficult for our dataset to represent the entire population of South Africa.

# <b>Introduction: </b>:

Every four years, the local municipalities in South Africa conduct a general evaluation roll that determines the municipal values of homes and the property rates that should be paid based on these values and ultimately, the price of the property. The property owners usually have objections to the property values determined by the municipality as these values are usually too low for a property value or too high for the prices that the owners must pay for using the municipality services. The property owners usually do not know how the municipality got to the property prices that come after the evaluation, as with the recent evaluation in where property owners experienced a hike in municipality bills. \[ [Private Property](https://www.privateproperty.co.za/advice/news/articles/property-valuations-shock-what-homeowners-can-do/6230)\]

In this project, we are investigating the factors that contribute to the property evalution prices using the dataset from Mbombela Municipality Evaluation Roll for 2010. This kind of analysis has not been done before. Useful site to extend this introduction:

1. [Schindlers](https://www.schindlers.co.za/2017/howdovalrollswork/)*****

## Research question:
 ### What factors dertermine the property value and to what extent do these factors contribute to the value of the property?
 
\*********
 
 
1. [Schindlers](https://www.schindlers.co.za/2017/howdovalrollswork/)


## Overview of methology:
A brief overview of the methodology is as follows:<br>
<br>
1) Read in data<br>
<br>
2) Data processing:<br>
&emsp;&emsp; Removing variables that are irrelavant to our question<br>
&emsp;&emsp; Removing Variables that are missing most of its values<br>
&emsp;&emsp; Recovering some variables values, null values, that can't be recovered<br>
&emsp;&emsp; Cleaning variables<br>
&emsp;&emsp; Changing Variable types<br>
<br>
3) Encode variables:<br>
&emsp;&emsp;        Encoding categorical variables
<br>
<br>
4) Exploratory Analyses:<br>
&emsp;&emsp;        Put Exploratory graphs<br>
&emsp;&emsp;        Calculate statistic figures:<br>
&emsp;&emsp;&emsp;&emsp;                eg. Mean, std, min, max and percentiles


## Section Contents overview:
1. [Data Description](#Data-Description)
2. [Data Wrangling](#Data-Wrangling)



# <b> Data Description: </b>

The dataset was obtained from [Open Africa:  Mbombela Municipality Property Valuation 2010 ](https://africaopendata.org/dataset/mbombela-municipality-property-valuation-2010) and downloaded on the 18 April 2021. The dataset contains over 57000 data points which have 49 features that describe some of the properties in Mbombela Municipality. 

#### Date collected:
The data was collected in Mbombela Municipal Council, in Mpumalanga, South Africa, in 2010. The last time the data modified was on December 21, 2017, 1:09 PM (UTC+02:00)
 
#### Dataset before cleaning:
The original dataset consists of 49 columns, 48 showing different factors contributing to the price of property in Mbombela. The description of these columns is given below:

- Township = The township in Mbombeni.
- Stand No = Stand number is used to identify the property
- Sect Title No = Section Title Number. Sectional Title, as a form of ownership (as per the Sectional Titles Act No.95 of 1986), emerged originally to permit parties to buy a piece or section of a larger property / building / development in a fashion where there ownership (or title) is protected (under Sectional Title law) and where there are clear rules and guidelines on how the overall property is managed, maintained and run. 
- StandDescrip = Stand Description.
- Owner Name = The owner of the property.
- Account No = Account Number. Account Number of the owner.
- Prop Category = Property Category
- TariffCode = Tariff Code.
- Physical Addr = Physical Address Showing the physical location of the property.
- Street No = Street Number. Identifying the street where the property is located
- Size = The size of the property measured in square meters
- Up-Level Size
- MarketValue = The price which the property is valued at. (Our Predictive Variable)
- Exemp Size
- Exemp Value
- Main Roll No = Main Roll Number. A Roll Number tells you everything you need to know about where a property is located. 
- Supp RollNo = Supplimentary roll number.Every property in every municipality should (hypothetically) be on a roll, but because properties are continuously coming into existence and ceasing to exist, new rolls are created (these are referred to as supplementary rolls) to include any properties that have not been previously recorded on another, prior, general roll.
- Owner Type = What type of owner is it (Private, municiplaity, Departmental and General)
- Valuation Date = When was the property evaluated in order to put a market value on it.
- Effect Date = The Effective Date for each Property Use Detail is the date that it was first valid for the property.
- Up-LevelInd
- Zone Code = Land zoning is how local governments, such as city councils, restrict the physical development and use of specific parcels of land. 
- Zone Descrip = Zone Description, Expanding on the codes and what it means. The Code descriptions are: 'BUSINESS 2', 'BUSINESS 1', 'PUBLIC GARAGE', 'RESIDENTIAL 4','SPECIAL', 'GOVERNMENT', 'MUNICIPAL', 'EXISTING PUBLIC ROADS', 'RESIDENTIAL 1', 'PUBLIC OPEN SPACE', 'BUSINESS 4', 'RESIDENTIAL 3', 'EDUCATIONAL', 'RESIDENTIAL 2', 'PRIVATE OPEN SPACE', 'INSTITUTIONAL', 'INDUSTRIAL 1', 'CHURCHES', 'INDUSTRIAL 3', 'INDUSTRIAL 2', 'TRANSNET', 'AGRICULTURE', 'BUSINESS 3', 'PARKING', 'CEMETRY', 'RECREATION', 'RAILWAY LINE(TRANSNET)', 'PRIVATE ROAD', 'RESIDENTIAL 5', 'GOVERNMENT & TELKOM', 'COMMERCIAL', 'KANYAMAZANE'
- Usage Code = Usage code is a code that details how the property will be used.
- UsageDescrip = Usage Description, Expanding on the codes and what it means. The Code descriptions are: 'BUSINESS 2', 'BUSINESS 1', 'PUBLIC GARAGE', 'RESIDENTIAL 4', 'SPECIAL', 'GOVERNMENT', 'MUNICIPAL', 'EXISTING PUBLIC ROADS', 'RESIDENTIAL 1', 'PUBLIC OPEN SPACE', 'BUSINESS 4', 'RESIDENTIAL 3', 'EDUCATIONAL', 'RESIDENTIAL BUSINESS', 'RESIDENTIAL 2', 'PRIVATE OPEN SPACE', 'DWELLINGS', 'INSTITUTIONAL', 'INDUSTRIAL 1', 'VACANT LAND SR', 'CHURCHES', 'INDUSTRIAL 3', 'PARKING', 'BED & BREAKFAST', 'INDUSTRIAL 2', 'TRANSNET', 'AGRICULTURE', 'BUSINESS 3', 'CEMETRY', 'RESIDENTIAL GUEST HOUSE', 'RESERVOIR', 'BUSINESS PREMISES', 'ROAD', 'RAILWAY LINE(TRANSNET)', 'AGRICULTURAL', 'PROPOSED NEW ROAD', 'RESIDENTIAL 5', 'VACANT LAND SINGLE RESIDENTIAL', 'POST OFFICE', 'COUNCIL VACANT RES 1 STANDS', 'FARM', 'SHOP', 'GOVERNMENT AND TELKOM', 'FACTORY', 'OFFICE', 'KANYAMASANE', 'SUBSTATION', 'CHURCH', 'VACANT LAND PLACES OF WORSHIP', 'CHRECHE', 'SCHOOL', 'PETROL FILLING STATION', 'CHICKEN HATCHERY', 'COMMUNITY CENTRE', 'ST DWELLING', 'ST RESIDENTIAL'
- Empty Stand Ind = Empty stand index that determine in the stand is empty or not.
- Valuation Type = The type of valuation method they used to value the property
- SGNumber
- Ward
- Cycle
- Meter Book
- Status = Property status refers to the situation of a property that was for sale, or is currently for sale.
- Val Cnt
- Postal Addr1 
- Postal Addr2
- Postal Addr3
- Postal Addr4
- Postal Addr5
- Postal Code
- BC Create Date
- BC Orig Date
- BC Altern Date
- BC Value
- BC Tariff
- Deed Date
- Deed No = Deed Number.
- Register Date = Date which the property was registered
- Register No = Property Registration Number. means the number issued by the City identifying the Application
- Capture Date

#### Aspects of Data Quality: 
- The data was retrieved from openAFRICA , openAFRICA is not a government portal. Instead, it’s a grassroots initiative, maintained by Code for Africa, as a public service.OpenAFRICA has additional support from Amazon Web Services (AWS) and the World Bank, both well known and accredited organisation thus making our source openAFRICA an accredible source.

- The data collection was done in 2010 and the evaluations of Market Value where done around the same time interval 2008 to 2010 making our Market values comparison fair because the amounts where not effected by other time factors like inflation. 

- The data was collected from the same region(Mbombeni)

<h3>Reading in data:</h3>

In [48]:
data = pd.read_excel('valuation-2010.xls')

### Validations:

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57252 entries, 0 to 57251
Data columns (total 49 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [55]:
data.describe()

,Account No,Size,Up-Level Size,MarketValue,Exemp Size,Exemp Value,Main Roll No,Supp RollNo,Valuation Date,Effect Date,...,Val Cnt,Postal Code,BC Create Date,BC Orig Date,BC Altern Date,BC Value,BC Tariff,Deed Date,Register Date,Capture Date
count,5.723900e+04,5.724600e+04,57246.0,5.724600e+04,57246.0,57246.0,57246.0,57246.000000,5.724600e+04,5.724600e+04,...,57246.000000,57227.000000,57227.0,57227.0,57227.0,0.0,0.0,5.722700e+04,5.722700e+04,5.722700e+04
mean,2.632920e+06,4.980989e+04,0.0,6.644221e+05,0.0,0.0,2.0,1.442913,2.009117e+07,2.009117e+07,...,1.000943,1900.546980,0.0,0.0,0.0,NaN,NaN,2.202494e+06,9.767406e+06,2.877971e+07
std,9.302737e+05,8.205704e+05,0.0,3.786126e+06,0.0,0.0,0.0,1.055363,2.034716e+03,2.034717e+03,...,0.040940,2221.613457,0.0,0.0,0.0,NaN,NaN,6.263835e+06,1.001051e+07,4.032805e+07
min,1.000855e+06,0.000000e+00,0.0,0.000000e+00,0.0,0.0,2.0,1.000000,2.009070e+07,2.009070e+07,...,1.000000,0.000000,0.0,0.0,0.0,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.114612e+06,3.520000e+02,0.0,5.600000e+04,0.0,0.0,2.0,1.000000,2.009070e+07,2.009070e+07,...,1.000000,1200.000000,0.0,0.0,0.0,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.506226e+06,5.420000e+02,0.0,2.400000e+05,0.0,0.0,2.0,1.000000,2.009070e+07,2.009070e+07,...,1.000000,1200.000000,0.0,0.0,0.0,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.062104e+06,1.108000e+03,0.0,7.000000e+05,0.0,0.0,2.0,1.000000,2.009070e+07,2.009070e+07,...,1.000000,1240.000000,0.0,0.0,0.0,NaN,NaN,0.000000e+00,2.004071e+07,2.009092e+07
max,1.564422e+07,9.327444e+07,0.0,6.600000e+08,0.0,0.0,2.0,6.000000,2.010012e+07,2.010012e+07,...,4.000000,9999.000000,0.0,0.0,0.0,NaN,NaN,2.009082e+07,2.202093e+07,1.000000e+08


In [5]:
data.head()


,Township,Stand No,Sect Title No,StandDescrip,Owner Name,Account No,Prop Category,TariffCode,Physical Addr,Street No,...,BC Create Date,BC Orig Date,BC Altern Date,BC Value,BC Tariff,Deed Date,Deed No,Register Date,Register No,Capture Date
0,NELSPRUIT TOWN,000000000000010000100000000,NaN,GED 1 ERF 1,NELSPRUIT HARDWARE,2218141.0,BUS,VABUS,PAUL KRUGER STREET,1,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,19810925.0,T46343/81,99999999.0
1,NELSPRUIT TOWN,00000000000001000RE00000000,NaN,RESTANT VAN ERF 1,MINTY IE & SONS PTY LTD,2218119.0,BUS,VABUS,ANDERSON STREET,36,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,19810925.0,T46344/81,99999999.0
2,NELSPRUIT TOWN,000000000000020000000000000,NaN,ERF 2,MINTYS UNIVERSAL STORES LTD,2218130.0,BUS,VABUS,ANDREW STREET,33,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,T96299/200,0.0
3,NELSPRUIT TOWN,000000000000030000000000000,NaN,ERF 3,LOWMIN PROPERTIES (PTY) LTD,2217568.0,BUS,VABUS,ANDERSON STREET,32,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,T96299/200,0.0
4,NELSPRUIT TOWN,000000000000040000000000000,NaN,ERF 4,NELMIN ERF 4 PROPERTIES (PTY) LTD,2040436.0,BUS,VABUS,ANDERSON STREET,30,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,20020814.0,T96299/200,20021029.0


# Data Wrangling:

<blockqoute>
In this next cell we drop several columns of the dataset either because we think they were irrelevent in determining the value of the property or the data had a lot of missing values or the information contained in the column was redundant as it was contained in another column.
</blockqoute>

In [2]:
#Columns to be removed
#Stand No, Sect Tittle, Account Number, TariffCode, Street No, Up-Level Size,
#Exemp Size, Main Roll No, Owner Type, Valuation Date, Up-levelInd, Usage Description
#Zone Description, SG Number, Status, Postal Address(1,2, 3, 4, 5), All columns with 'BC', Postal Code,
#Deed No, Register No, Valuation Date, Capture Date 
# Stand No, Sect Tittle, Account Number, TarrifCode(Because it is dependent ), Street No,Up-Level Size,  Exemp Size,Exemp Value,  
#Main Roll No, Owner Type(dependent on), Valuation date, Up-LevelInd, usage description, Zone Description, SGNumber, 
# Status, (Postal Adrress1, 2, 3, 4, 5), BC Date, Postal Code, Deed No,(All BC Values), Register No.
#Valuation Date(date time type), Capture Date
# Usage Code: Discard later if needed
# What does Empty Stand Ind - indicate?(remove if possible)
# Investigate the meaning of Valuation Type Values
#More info on: Meter Book
# Verify if Postal Code matches with the township
# 
data_new = data.drop(columns = ['Stand No', 'StandDescrip','Sect Title No', 'Account No', 'TariffCode',
       'Street No', 'Up-Level Size','Exemp Size',
       'Exemp Value', 'Main Roll No', 'Owner Type',
       'Valuation Date', 'Up-LevelInd','BC Create Date',
       'BC Orig Date', 'BC Altern Date', 'BC Value','BC Tariff',
       'Zone Descrip', 'UsageDescrip','SGNumber','Status',
       'Postal Addr1', 'Postal Addr2', 'Postal Addr3',
       'Postal Addr4', 'Postal Addr5', 'Postal Code',
       'Deed Date', 'Deed No', 'Register Date', 'Register No'])
data_new = data_new.drop(columns = [data_new.columns[-1]])
##Number of columns in the oriiginal data set
num_cols_orig = len(data.columns)
new_num_cols = len(data_new.columns)
left_cols= num_cols_orig - new_num_cols
#Capute date is not properly named, so we must remove it using the indexs
print('After cleaning our data, we were able to remove', left_cols, 'And were left with: ', new_num_cols)
# print('\nWe then set our MarketValue as our independed variable, the one we want to predict and moved it to the last columns for easy accessing')
# temp_col = data_new['MarketValue']
# data_new = data_new.drop(columns = ['MarketValue'])
# data_new['MarketValue'] = temp_col
print('\n This is our dataset after perfoming the above operations')
data_new.head(3)

NameError: name 'data' is not defined

Now that we have removed all the dirty columns by dropping the them, we continued to clean the dataset that was left by cleaning the Nan values by either trying to recover lost data using the current existing data or removing the Nan data if it was unrecoverable.

## Wrangling from column 0 to column 5:

We clean the remain dataset in portion to avoid the errors and make it easy to manage and trace mistakes in the data. Beginning with columns from 1 to column 5

In [3]:
df = pd.DataFrame()
df = data_new.copy()
df.head()

NameError: name 'data_new' is not defined

In [4]:
print('Showing the null values found in the above dataframe for each columns\n')
print('Township contains ', df['Township'].isnull().sum(), 'null values')          #printing null values of Township
print('Owner Name column contains', df['Owner Name'].isnull().sum(), 'null values')#printing null values of Owner Name
print('Prop Category contains', df['Prop Category'].isnull().sum(), 'null values') #printing null values of Prop Category
print('Physical Addr contains', df['Physical Addr'].isnull().sum(), 'null values') #printing null values of Physical Addr
print('Size column contains', df['Size'].isnull().sum(), 'null values')            #printing null values of Size
print('MarketValue contains', df['MarketValue'].isnull().sum(), 'null values')     #printing null values of MarketValue

Showing the null values found in the above dataframe for each columns



KeyError: 'Township'

We will start with the columns that contains least number of null/Nan. 

We start by the Prop Category column, and encode the different attributes using numbers, the we encoded the Nan in the Owners column with "Other" to represent the owners we do not have details of.

We filled the Nan values in the columns 'Owner Name' and 'Physical Addr' with the word 'Other' and 'Unknown' respectively as they was no method to recover them.

In [5]:
#encode prop category
#1 BUS and #2 RES #3 GOV #4 MUN #5 POW #6 OTH #7 PSI #8 AG #9 RUR
df['Prop Category'] = np.where(df['Prop Category'] == 'BUS', 1 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'RES', 2 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'GOV', 3 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'MUN', 4 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'POW', 5 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'OTH', 6 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'PSI', 7 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'AG',  8 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'RUR', 9 ,df['Prop Category'])

KeyError: 'Prop Category'

In [6]:
#encode missing owners name with 'other'
df['Owner Name'] = df['Owner Name'].fillna('Other')
df['Owner Name'].isnull().sum()

KeyError: 'Owner Name'

In [7]:
#encode missing values for physical address
df['Physical Addr'] = df['Physical Addr'].fillna('Unknown')
df['Physical Addr'].isnull().sum()

KeyError: 'Physical Addr'

In [8]:
physical_addr = df['Physical Addr'].tolist()
streets  = []
street_num  = 1
for i, street in enumerate(physical_addr):
    if street not in streets:
        streets.append(street)
        physical_addr[i] = streets.index(street) + 1
    else:
        physical_addr[i] = streets.index(street) + 1
df['Physical Addr'] = physical_addr

KeyError: 'Physical Addr'

In [9]:
df['Prop Category']

KeyError: 'Prop Category'

In [10]:
print('This is the dataframe after cleaning the columns 0-5')
##Store this new cleaned to df_clean
df_clean = df.copy()
df.head()


This is the dataframe after cleaning the columns 0-5


""


## Data Wrangling on column 6-12:

In [11]:
#Data wrangling of dataset from Column 7 to Coulumn 13
data_section = df_clean.iloc[:,6:12]
data_section

""


Noting that there is only N and Nan values in the 'Empty Stand Ind' column, we could have decided that the Nan represented YES but that assumption was far fetched as we cannot conclude that the stand is empty or not give as there may be other factors affecting this variable, as a result we dropped this column and have the result shown below.


In [12]:
#There is only N and null values in 'Empty Stand Ind'. The question is 'is null equal to yes?', this assumption is far fetched thus might be incorrect, based on that we take out this column

##For debugging purposes
try:
    df_clean = df_clean.drop(columns = 'Empty Stand Ind')
except:
    print('The column Epmty Stand Ind has been removed already')
data_section = df_clean.iloc[:,6:12]
data_section

The column Epmty Stand Ind has been removed already


""


In [13]:
print('This is the number of columns with missing values')
#Checking columns with null values
ncols_missing = data_section.isnull().any(axis=0).sum()
print(ncols_missing)
print('This is the number of rows with missing values')
#Checking rows with null values
nrows_missing = data_section.isnull().any(axis=1).sum()
print(nrows_missing)

This is the number of columns with missing values
0
This is the number of rows with missing values
0


In [14]:
#All 6 columns have missing values
#6 rows have missing values

#For seing which rows index are null in respect to each column.
print(np.where(data_section['Supp RollNo'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Supp RollNo")
print(np.where(data_section['Effect Date'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Effect Date")
print(np.where(data_section['Zone Code'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Zone Code")
print(np.where(data_section['Usage Code'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Usage Code")
print(np.where(data_section['Valuation Type'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Valuation Type")
print(np.where(data_section['Ward'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Ward")

KeyError: 'Supp RollNo'

In [15]:
print(df_clean.loc[[30400, 40754, 40767, 40800, 40815, 40834]])
#All the rows with missing values are removed
data_new = df_clean.drop([30400, 40754, 40767, 40800, 40815, 40834]);

KeyError: "None of [Int64Index([30400, 40754, 40767, 40800, 40815, 40834], dtype='int64')] are in the [index]"

In [16]:
print('This is the number of columns with missing values')
data_section = df_clean.iloc[:,7:12]
#Checking columns with null values
ncols_missing = data_section.isnull().any(axis=0).sum()
print(ncols_missing)
print('This is the number of rows with the missing values')
#Checking rows with null values
nrows_missing = data_section.isnull().any(axis=1).sum()
print(nrows_missing)

This is the number of columns with missing values
0
This is the number of rows with the missing values
0


In [17]:
#Making my dictionaries
dic = pd.Series(data_section['Zone Code'].unique())
dic.index = dic.index + 1
dic = pd.Series(dic.index.values, index = dic)

dic2 = pd.Series(data_section['Usage Code'].unique())
dic2.index = dic2.index + 1
dic2 = pd.Series(dic2.index.values, index = dic2)

dic3 = pd.Series(data_section['Valuation Type'].unique())
dic3.index = dic3.index + 1
dic3 = pd.Series(dic3.index.values, index = dic3)

KeyError: 'Zone Code'

In [18]:
#Encoding based on the dictionaries
lookup_table = dict(dic)

def lookup_customer(val):
    return lookup_table.get(val, np.nan)


data_section['Zone Code'] = data_section['Zone Code'].apply(lambda x: lookup_customer(x));

lookup_table = dict(dic2)
data_section['Usage Code'] = data_section['Usage Code'].apply(lambda x: lookup_customer(x));

lookup_table = dict(dic3)
data_section['Valuation Type'] = data_section['Valuation Type'].apply(lambda x: lookup_customer(x));

data_section
#df_new.head(20)

NameError: name 'dic' is not defined

In [19]:
###Supp RollNo is Supplimentary roll number
##Every property in every municipality should (hypothetically) be on a roll, but because properties are continuously coming into existence and ceasing to exist, new rolls are created (these are referred to as supplementary rolls) to include any properties that have not been previously recorded on another, prior, general roll.
data_section.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

In [20]:
#convert column Effect date to date time
data_section['Effect Date'] = pd.to_datetime(data_section['Effect Date'], format='%Y%m%d') 
data_section

KeyError: 'Effect Date'

In [21]:
print('The final cleaned dataframe')
df_clean.iloc[:,7:12] = data_section
df_clean

The final cleaned dataframe


""


In [22]:
#Note that ERF is afrikaans and it means inherit
#Restant mean remaining when translated from afrikaans to english
#Restant van Erf therefore means remaing of land

## Cleaning Columns 13-17: